# STEP 0: Use GPU

In [1]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()

True

# STEP 1: Git clone project

In [2]:
# Git clone your project

!git clone https://github.com/yoyoLin427/Nuclei-segmentation.git

fatal: destination path 'Nuclei-segmentation' already exists and is not an empty directory.


# STEP 2: Install detectron2

After installation, you may need to "restart runtime" in Colab.


In [3]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.10 ; cuda:  cu111


In [4]:
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

#exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


# step3: prepare testing data

In [5]:
!unzip /content/Nuclei-segmentation/dataset_test.zip

Archive:  /content/Nuclei-segmentation/dataset_test.zip
   creating: test/
   creating: test/.ipynb_checkpoints/
  inflating: test/.ipynb_checkpoints/TCGA-50-5931-01Z-00-DX1-checkpoint.png  
  inflating: test/.ipynb_checkpoints/TCGA-AY-A8YK-01A-01-TS1-checkpoint.png  
  inflating: test/.ipynb_checkpoints/TCGA-G9-6336-01Z-00-DX1-checkpoint.png  
  inflating: test/.ipynb_checkpoints/TCGA-G9-6348-01Z-00-DX1-checkpoint.png  
  inflating: test/TCGA-50-5931-01Z-00-DX1.png  
  inflating: test/TCGA-A7-A13E-01Z-00-DX1.png  
  inflating: test/TCGA-AY-A8YK-01A-01-TS1.png  
  inflating: test/TCGA-G2-A2EK-01A-02-TSB.png  
  inflating: test/TCGA-G9-6336-01Z-00-DX1.png  
  inflating: test/TCGA-G9-6348-01Z-00-DX1.png  
  inflating: test_img_ids.json       


In [6]:
import json 

with open('/content/test_img_ids.json') as f:
    data = json.load(f)

for i in range(len(data)):
  print(data[i]['file_name'])

TCGA-A7-A13E-01Z-00-DX1.png
TCGA-50-5931-01Z-00-DX1.png
TCGA-G2-A2EK-01A-02-TSB.png
TCGA-AY-A8YK-01A-01-TS1.png
TCGA-G9-6336-01Z-00-DX1.png
TCGA-G9-6348-01Z-00-DX1.png


# step4: start predicting

In [7]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [8]:
import pycocotools._mask as _mask
def encode(bimask):
    if len(bimask.shape) == 3:
        return _mask.encode(bimask)
    elif len(bimask.shape) == 2:
        h, w = bimask.shape
        return _mask.encode(bimask.reshape((h, w, 1), order="F"))[0]

def area(rleObjs):
    if type(rleObjs) == list:
        return _mask.area(rleObjs)
    else:
        return _mask.area([rleObjs])[0]

def toBbox(rleObjs):
    if type(rleObjs) == list:
        return _mask.toBbox(rleObjs)
    else:
        return _mask.toBbox([rleObjs])[0]

### download my weight

In [9]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1RUtuR7hJozGfQciULRCSds9DV3XJcauS',
                                      dest_path='/model_final.pth',
                                      unzip=False)


In [10]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("balloon_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.005   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [11]:
ans = []
for i in range(len(data)):
  print(data[i]['file_name'])

  image_id = i + 1
  segmentation_list = []
  bbox_list = []
  score_list = []

  im = cv2.imread('/content/test/'+data[i]['file_name'])
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format

  for s in outputs["instances"].scores:
    score_list.append(float(s))

  masks = outputs["instances"].pred_masks.cpu()
  for mask in masks:
    mask = mask.long()
    mask = mask.numpy()
    mmask = cv2.normalize(mask, None, 0, 255, cv2.NORM_MINMAX)
    m = np.asfortranarray(np.uint8(mmask))
    seg = encode(m)
        
    segcounts = seg["counts"].decode("UTF-8")
    tmpseg = {
            "size": seg["size"],
            "counts": segcounts,
        }
    segmentation_list.append(tmpseg)
    box = toBbox(seg)
    tmpb = []
    for b in box:
      tmpb.append(float(b))
    bbox_list.append(tmpb)
  for j in range(len(bbox_list)):
    k = {
            "image_id": int(image_id),
            "bbox": bbox_list[j],
            "score": score_list[j],
            "category_id": int(1),
            "segmentation": segmentation_list[j],
        }
    ans.append(k)

TCGA-A7-A13E-01Z-00-DX1.png


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


TCGA-50-5931-01Z-00-DX1.png
TCGA-G2-A2EK-01A-02-TSB.png
TCGA-AY-A8YK-01A-01-TS1.png
TCGA-G9-6336-01Z-00-DX1.png
TCGA-G9-6348-01Z-00-DX1.png


In [12]:
with open("answer.json", "w") as outfile:
  json.dump(ans, outfile, indent=4)